## Loading Libraries

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
import random
from datetime import datetime, timedelta
import os

Create synthetic dataset 

In [3]:
# Define the products and regions
products = ['Product A', 'Product B', 'Product C', 'Product D', 'Product E']
regions = ['Region 1', 'Region 2', 'Region 3', 'Region 4', 'Region 5']

# Set the random seed
random.seed(123)

# Generate random sales data
sales_data = []
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)
for i in range(1000):
    product = random.choice(products)
    region = random.choice(regions)
    sale_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    sales = np.round(random.uniform(1000, 10000), 2)
    sales_data.append([product, region, sale_date, sales])

# Convert the sales data to a Pandas DataFrame
columns = ['Product', 'Region', 'Sale Date', 'Sales']
df = pd.DataFrame(sales_data, columns=columns)

# Save the DataFrame to a CSV file
df.to_csv('sales_data.csv', index=False)


In [4]:
# Define the name of the database and the name of the table
database = 'data/sales.db'
table = 'sales_data'

# Create the data folder if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

# Connect to the database
conn = sqlite3.connect(database)

# Check if the table already exists
cursor = conn.cursor()
cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table}'")
table_exists = cursor.fetchone() is not None

# If the table doesn't exist, create it
if not table_exists:
    df.to_sql(table, conn, index=False)

# If the table already exists, append the new data to it
else:
    df.to_sql(table, conn, index=False, if_exists='append')

# Close the connection
conn.close()


In [5]:
%load_ext sql

In [6]:
%sql sqlite:///data/sales.db

'Connected: @data/sales.db'

In [7]:
%%sql 

select * from sales_data limit 10

 * sqlite:///data/sales.db
Done.


Product,Region,Sale Date,Sales
Product A,Region 3,2022-02-14 00:00:00,7920.61
Product C,Region 1,2022-01-20 00:00:00,4412.12
Product E,Region 3,2022-06-24 00:00:00,8668.78
Product B,Region 2,2022-06-22 00:00:00,6048.66
Product B,Region 2,2022-01-01 00:00:00,9153.15
Product A,Region 5,2022-07-13 00:00:00,1628.89
Product C,Region 4,2022-02-22 00:00:00,9148.59
Product A,Region 2,2022-03-06 00:00:00,8110.65
Product A,Region 3,2022-08-09 00:00:00,6160.67
Product C,Region 4,2022-01-19 00:00:00,7919.15


In [8]:
%%sql
select product, sum(sales)
from sales_data
where region = 'Region 1'
group by product

 * sqlite:///data/sales.db
Done.


Product,sum(sales)
Product A,441877.06000000006
Product B,460947.1599999999
Product C,521125.4800000001
Product D,434785.93999999994
Product E,463864.43999999994


In [9]:
%%sql 
SELECT
    region,
    SUM(CASE WHEN product = 'Product A' THEN sales ELSE 0 END) AS product_a_sales,
    SUM(CASE WHEN product = 'Product B' THEN sales ELSE 0 END) AS product_b_sales,
    SUM(CASE WHEN product = 'Product C' THEN sales ELSE 0 END) AS product_c_sales,
    SUM(CASE WHEN product = 'Product D' THEN sales ELSE 0 END) AS product_d_sales,
    SUM(CASE WHEN product = 'Product E' THEN sales ELSE 0 END) AS product_e_sales
FROM
    sales_data
GROUP BY
    region;

 * sqlite:///data/sales.db
Done.


Region,product_a_sales,product_b_sales,product_c_sales,product_d_sales,product_e_sales
Region 1,441877.06000000006,460947.1599999999,521125.4800000001,434785.93999999994,463864.43999999994
Region 2,550697.7000000002,335941.51999999996,528565.88,374354.31999999995,364844.70000000007
Region 3,336101.46,501694.10000000015,433529.9399999999,362397.45999999996,513220.87999999995
Region 4,337914.32,557967.66,473402.30000000005,451388.74000000017,385389.73999999993
Region 5,423003.2000000002,530907.1000000004,392408.92,459287.12,404601.7600000001


In [10]:
%%sql

WITH product_sales_by_region AS (
    SELECT
        region,
        product,
        SUM(sales) AS total_sales
    FROM
        sales_data
    GROUP BY
        region,
        product
), top_selling_product_by_region AS (
    SELECT
        region,
        product,
        total_sales,
        RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) AS rank
    FROM
        product_sales_by_region
)
SELECT
    region,
    product,
    total_sales
FROM
    top_selling_product_by_region
WHERE
    rank = 1;


 * sqlite:///data/sales.db
Done.


region,product,total_sales
Region 1,Product C,521125.4800000001
Region 2,Product A,550697.7000000002
Region 3,Product E,513220.87999999995
Region 4,Product B,557967.66
Region 5,Product B,530907.1000000004
